<a href="https://colab.research.google.com/github/JCaballerot/Deep_learning_program/blob/main/Topicos_avanzados/time_series_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><font size="5"> Time Series starter dataset </font></h1>

Nuestro objetivo es construir un modelo predictivo robusto utilizando redes neuronales de Memoria a Corto y Largo Plazo (LSTM, por sus siglas en inglés), una clase especial de redes neuronales recurrentes diseñadas para capturar dependencias temporales a largo plazo de manera efectiva. Las redes LSTM son particularmente adecuadas para esta tarea debido a su capacidad para recordar información por periodos prolongados, lo que las hace ideales para analizar series temporales.

In [10]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np

### 1. Carga y Preparación de los Datos


Se importa la biblioteca pandas para manipular el conjunto de datos.
El conjunto de datos se carga desde un archivo CSV. Los datos faltantes se eliminan con dropna() para asegurar la calidad del entrenamiento.

In [24]:

df = pd.read_csv('Month_Value_1.csv')
df = df.dropna()

Se utiliza matplotlib para graficar la serie temporal de Average_cost a lo largo del período. Esto permite visualizar tendencias, patrones y posibles anomalías en los datos.
Las etiquetas del eje X se rotan para mejorar la legibilidad.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Plotting the time series data
plt.figure(figsize=(10,4))
plt.plot(df['Period'], df['Average_cost'], label='Average Cost Over Time')

# Rotate the x labels slightly
plt.xticks(rotation=45)

plt.title('Average Cost Time Series')
plt.xlabel('Period')
plt.ylabel('Average Cost')
plt.legend()
plt.tight_layout()  # Adjust layout to prevent clipping of ylabel
plt.show()

### 2. Preprocesamiento de Datos para la LSTM



Los datos se convierten en un formato adecuado para alimentar la red LSTM. Esto implica crear secuencias de un número específico de pasos de tiempo como entrada y el valor siguiente como etiqueta para la predicción.
Se redefine la forma de los datos de entrada para que coincida con lo que espera la red LSTM, [muestras, pasos de tiempo, características].

In [ ]:
np.array(df.Average_cost)

In [ ]:

# Serie de ejemplo
data = np.array(df.Average_cost)

# Preparar los datos para la LSTM
def create_dataset(data, steps):
    X, y = [], []
    for i in range(len(data)-steps):
        X.append(data[i:(i+steps)])
        y.append(data[i+steps])
    return np.array(X), np.array(y)

# Definir el número de pasos de tiempo
steps = 3
X, y = create_dataset(data, steps)

# Reshape de X para que sea [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))


### 3. Construcción del Modelo LSTM



Se construye un modelo secuencial que incluye dos capas LSTM y una capa densa al final.
La primera capa LSTM tiene return_sequences=True para pasar la secuencia a la siguiente capa LSTM. La segunda capa LSTM ayuda a extraer características más complejas y no necesita devolver la secuencia completa.
La capa densa final se usa para realizar la predicción del valor futuro en la serie.

In [ ]:

# Construir el modelo LSTM
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(steps, 1)))  # Capa LSTM con return_sequences=True
model.add(LSTM(50, activation='relu'))  # Segunda capa LSTM, no necesita return_sequences
model.add(Dense(1))  # Capa densa para la predicción
model.compile(optimizer='adam', loss='mse')



El modelo se entrena con los datos preparados, utilizando el error cuadrático medio (MSE) como función de pérdida y el optimizador Adam.
Se eligen 200 épocas para el entrenamiento con verbose=0 para minimizar la salida de registro durante este proceso.

In [ ]:

# Entrenar el modelo
model.fit(X, y, epochs = 200, verbose=0)

### 4. Predicción de Valores Futuros

Se utiliza el modelo entrenado para predecir los siguientes 10 valores en la serie. Para esto, se alimenta el modelo con los últimos datos conocidos y se van agregando las predicciones a la entrada para las predicciones subsiguientes.
Esto demuestra cómo el modelo puede ser usado para proyecciones a corto plazo basadas en datos históricos.

In [ ]:


# Hacer predicciones
X_input = data[-steps:]
X_input = X_input.reshape((1, steps, 1))

predictions = []

for i in range(10):  # Predecir los siguientes 10 puntos
    pred = model.predict(X_input, verbose=0)
    predictions.append(pred[0,0])
    X_input = np.append(X_input[0,1:], pred)
    X_input = X_input.reshape((1, steps, 1))

predictions = np.array(predictions)
predictions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Plotting the time series data
plt.figure(figsize=(8,4))
plt.plot(df.Average_cost.tolist() + list(predictions), label='Average Cost Over Time')

# Rotate the x labels slightly
plt.xticks(rotation=45)

plt.title('Average Cost Time Series')
plt.xlabel('Period')
plt.ylabel('Average Cost')
plt.legend()
plt.tight_layout()  # Adjust layout to prevent clipping of ylabel
plt.show()

---
## Gracias por completar este laboratorio!